In [1]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time

In [2]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return weights.astype(np.float32), bias.astype(np.float32)

In [5]:
def conv2d(self,inp,kernels,biases,stride=[1,1],padding=0):     #padding=(ksz-1)/2 for same shape in stride 1
    #inp[batches,row,col,d],kernels(d,ksz,ksz,num_ker),biases[1,num_ker],stride[row,col]
    inp=inp.transpose(0,3,1,2)  #inp[batches,d,row,col]
    output=[]
    ksz=kernels.shape[1]
    num_ker=kernels.shape[3]
    if not padding:                         #take care of padding in backprop too
        padding=(ksz-1)//2                  #currently don't give 'even' ksz
    out_row,out_col=((inp.shape[2]-ksz+2*padding)//stride[0]+1),((inp.shape[3]-ksz+2*padding)//stride[1]+1)
    batches,d,row,col=inp.shape
    row+=2*padding
    col+=2*padding
    padded=np.zeros((batches,d,row,col))
    padded[:,:,padding:-padding,padding:-padding]=inp
    # Take all windows into a matrix       # TO-DO: Cache up these calculations
    window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
    slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
    ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
    kern = kernels.reshape(-1,num_ker)
    for img in padded:      #img[d,row,col]     #TODO: MAKE THIS FOR LOOP INTO SINGLE NUMPY OPERATION
        # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
        out=(np.dot(np.take(img, ind), kern))
        out=(out+biases).reshape(out_row,out_col,num_ker)
        output.append(out)
#     output=np.array([(np.dot(np.take(i,ind),kern)+biases) for i in padded]).reshape(batches,out_row,out_col,num_ker)
#     bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()		#for batches
#     output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+biases).reshape(batches,out_row,out_col,num_ker)
    return np.array(output) #output[batches,out_row,out_col,num_ker]

In [ ]:
%timeit conv0=conv2d(1,inp,w0,b0)

In [ ]:
%timeit conv0=nn.conv2d(inp,w0,b0)

In [ ]:
conv0=conv2d(1,inp,w0,b0)

In [ ]:
%timeit d_X_inp,d_w0,d_b0=nn.conv2d_back(errors=conv0,inp=inp,kernels=w0,biases=b0)

In [ ]:
conv0.shape

In [61]:
inp=np.random.randint(0,9,(1,4,4,1)).astype(np.float32)
w0,b0=init_kernel_bias(num_inp_channels=1,kernel_size=3,num_kernels=1)
w0.shape

(1, 3, 3, 1)

In [58]:
inp=np.random.randint(0,9,(128,32,32,64)).astype(np.float32)
w0,b0=init_kernel_bias(num_inp_channels=64,kernel_size=3,num_kernels=32)
w0.shape

(64, 3, 3, 32)

In [62]:
dilation=[1,1]

In [63]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=1
stride=[2,2]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding-(ksz-1)*(dilation[0]-1))//stride[0]+1),((ipp.shape[3]-ksz+2*padding-(ksz-1)*(dilation[1]-1))//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col),dtype=np.float32)
padded[:,:,padding:-padding,padding:-padding]=ipp

In [64]:
%timeit padded[:,:,padding:-padding:1,padding:-padding:1]=ipp

976 ns ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [68]:
img=padded[0]
row,col,d

(6, 6, 1)

In [69]:
out_row=2
out_col=2

In [70]:
dksz=ksz+(ksz-1)*(dilation[0]-1)

In [75]:
# %%timeit
window=(np.arange(dksz,step=dilation[0])[:,None]*row+np.arange(dksz,step=dilation[1])).ravel()+np.arange(d)[:,None]*row*col+6
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker)
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [76]:
img.shape,row,col,out_row,out_col,ksz

((1, 6, 6), 6, 6, 2, 2, 3)

In [77]:
window

array([[ 6,  7,  8, 12, 13, 14, 18, 19, 20]])

In [78]:
slider

array([[ 0,  1,  2,  3],
       [ 6,  7,  8,  9],
       [12, 13, 14, 15],
       [18, 19, 20, 21]])

In [79]:
ind

array([[ 6,  7,  8, 12, 13, 14, 18, 19, 20],
       [ 8,  9, 10, 14, 15, 16, 20, 21, 22],
       [18, 19, 20, 24, 25, 26, 30, 31, 32],
       [20, 21, 22, 26, 27, 28, 32, 33, 34]])

In [17]:
ind.size*batches,ind.shape

(147456, (4, 288))

In [18]:
# padded=np.arange(padded.size,dtype=np.float32).reshape(padded.shape)

In [19]:
padded.shape

(128, 32, 66, 66)

In [ ]:
coled=np.empty(ind.size*batches,dtype=np.float32).reshape(-1,d*ksz*ksz)

In [69]:
batches,out_row,out_col

(2, 3, 3)

In [70]:
coled.shape

(18, 4)

In [ ]:
%%time
ctake.take(c_void_p(padded.ctypes.data),c_void_p(ind.ctypes.data),c_void_p(coled.ctypes.data),c_int(1),c_int(padded[0].size),c_int(ind.size),c_int(4))

In [72]:
a=conv2d(32,kernel_size=3,stride=[2,2],input_shape=(32,32,3))

In [55]:
a.shape

(None, 4, 4, 1)

In [76]:
%%time
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)+b0
ans=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 386 µs, sys: 0 ns, total: 386 µs
Wall time: 309 µs


In [77]:
output.shape,ans.shape

((2, 9, 1), (2, 3, 3, 1))

In [25]:
%%time
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=img.take(ind).dot(kern)
if b0 is not 0:
    output+=b0
ans=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 106 µs, sys: 0 ns, total: 106 µs
Wall time: 80.3 µs


In [28]:
%%timeit
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)
output+=b0
ans2=output.reshape(batches,out_row,out_col,num_ker)

1.24 s ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
%timeit np.array([(np.dot(np.take(i,ind),kern)) for i in padded]).reshape(batches,out_row,out_col,num_ker)

1.39 s ± 80.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [95]:
%timeit (np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

928 ms ± 54.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [96]:
%timeit (np.dot(padded.take(bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

931 ms ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
np.take(img,ind).shape,kern.shape

((4096, 288), (288, 128))

In [24]:
(np.dot(np.take(img,ind), kern)).shape

(4096, 128)

In [26]:
ans=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+b0).reshape(batches,out_row,out_col,num_ker)

In [27]:
ans.shape

(128, 64, 64, 128)

In [1]:
from nnet.layers import Layer,conv2d,conv2dtranspose

Seed: 436


In [2]:
import cupy as cp
import numpy as np

In [3]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return weights.astype(np.float32), bias.astype(np.float32)

In [4]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=64)

In [5]:
inp=np.random.randn(128,60,60,32).astype(np.float32)

In [6]:
inp.shape,w0.shape

((128, 60, 60, 32), (32, 3, 3, 64))

In [7]:
inpd=cp.asarray(inp.transpose(0,3,1,2))
w0d=cp.asarray(w0)

In [8]:
ch, kh, kw, nk = w0d.shape
bt, ch, h, w = inpd.shape
sy,sx = (1,1)
ph,pw = (1,1)
dy,dx = (1,1)
out_h,out_w = (h,w)
col = cp.empty((bt, ch, kh, kw, out_h, out_w), dtype=inpd.dtype)
im2col = cp.ElementwiseKernel(
    'raw T inp, int32 row, int32 col, int32 out_row, int32 out_col,'
    'int32 kh, int32 kw, int32 sy, int32 sx, int32 ph, int32 pw,'
    'int32 dy, int32 dx',
    'T coled',
    '''
       int c0 = i / (kh * kw * out_row * out_col);   // select channel
       int ky = i / (kw * out_row * out_col) % kh;   // select kernel y
       int kx = i / (out_row * out_col) % kw;        // select kernel x
       int out_y = i / out_col % out_row;            // select output y
       int out_x = i % out_col;                    // select output x
       int in_y = ky * dy + out_y * sy - ph;
       int in_x = kx * dx + out_x * sx - pw;
       if (in_y >= 0 && in_y < row && in_x >= 0 && in_x < col) {    // if in image bounds
         coled = inp[col * (in_y + row * c0) + in_x]; // choose pixel
       } else {
         coled = 0;                                // pad with 0
       }
    ''',
    'im2col')

In [9]:
inpd.shape,col.shape,w0d.shape

((128, 32, 60, 60), (128, 32, 3, 3, 60, 60), (32, 3, 3, 64))

In [14]:
%%time
col=im2col(inpd.reduced_view(),
              h, w, out_h, out_w, kh, kw, sy, sx, ph, pw, dy, dx, col)
outd=cp.tensordot(col, w0d, ((1, 2, 3), (0, 1, 2)))
cp.cuda.Stream.null.synchronize()

CPU times: user 458 ms, sys: 2.54 ms, total: 461 ms
Wall time: 461 ms


In [14]:
outd.shape,col.shape

((128, 60, 60, 64), (128, 32, 3, 3, 60, 60))

In [16]:
# del col

In [16]:
grads=outd.transpose(0,3,1,2)
wtd=w0d#.transpose(3,0,1,2)

In [17]:
gcol=col.reshape(32, 3, 3, 128, 60, 60)

In [34]:
%%time
gcol=cp.tensordot(wtd,grads,(3,1))
cp.cuda.Stream.null.synchronize()

CPU times: user 72 ms, sys: 3.5 ms, total: 75.5 ms
Wall time: 74.1 ms


In [43]:
del gcol

In [22]:
gcol.shape

(32, 3, 3, 128, 60, 60)

In [23]:
%%time
gcol=cp.ascontiguousarray(cp.moveaxis(gcol,3,0))           # REMOVE THIS SOMEHOW
# gcol=gcol.transpose(3,0,1,2,4,5)
# gcol=cp.ascontiguousarray(gcol)
cp.cuda.Stream.null.synchronize()

CPU times: user 141 ms, sys: 308 µs, total: 141 ms
Wall time: 139 ms


In [24]:
gcol.shape

(128, 32, 3, 3, 60, 60)

In [25]:
gcol.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True

In [ ]:
n, ch, kh, kw, out_h, out_w = gcol.shape
h, w = out_h, out_w
img = cp.empty((n, ch, h, w), dtype=gcol.dtype)
col2im=cp.ElementwiseKernel(
    'raw T coled, int32 row, int32 col, int32 out_row, int32 out_col,'
    'int32 kh, int32 kw, int32 sy, int32 sx, int32 ph, int32 pw,'
    'int32 dy, int32 dx',
    'T inp',
    '''
       int c0 = i / (row * col);
       int y  = i / col % row;
       int x  = i % col;
       T val = 0;
       for (int ky = 0; ky < kh; ++ky) {
         int out_y = (y + ph - ky * dy);
         if (0 > out_y || out_y >= out_row * sy) continue;
         if (out_y % sy != 0) continue;
         out_y /= sy;
         for (int kx = 0; kx < kw; ++kx) {
           int out_x = (x + pw - kx * dx);
           if (0 > out_x || out_x >= out_col * sx) continue;
           if (out_x % sx != 0) continue;
           out_x /= sx;
           int k = out_y + out_row * (kx + kw * (ky + kh * c0));
           val = val + coled[out_x + out_col * k];
         }
       }
       inp = val;
    ''',
    'col2im')

In [77]:
%%time
img=col2im(gcol.reduced_view(),
              h, w, out_h, out_w, kh, kw, sy, sx, ph, pw, dy, dx, img)
cp.cuda.Stream.null.synchronize()

CPU times: user 89.8 ms, sys: 11 µs, total: 89.8 ms
Wall time: 88.9 ms


In [78]:
img.shape

(128, 32, 60, 60)

In [79]:
%%time
img=cp.ascontiguousarray(img.transpose(0,2,3,1).transpose(0,3,1,2))
cp.cuda.Stream.null.synchronize()

CPU times: user 89 µs, sys: 23 µs, total: 112 µs
Wall time: 117 µs


In [74]:
ct=conv2dtranspose(input_shape=(60,60,64),kernel_size=3,stride=[1,1],kernels=w0[:,::-1,::-1,:].transpose(3,1,2,0),biases=0)

In [75]:
cto=ct.forward(outd.get())

In [80]:
np.allclose(img.transpose(0,2,3,1).get(),cto,atol=1e-07)

True

In [145]:
gcol.nbytes/1024/1024

506.25

In [ ]:
from nnet_gpu.layers import conv2dtranspose

In [30]:
c=conv2d(input_shape=(60,60,32),kernels=w0,biases=0)

In [31]:
%%time
out=c.forward(inp)

CPU times: user 1.26 s, sys: 275 ms, total: 1.54 s
Wall time: 519 ms


In [55]:
outd.shape,out.shape

((128, 60, 60, 64), (128, 60, 60, 64))

In [139]:
np.allclose(out,outd.get(),atol=1e-06/2)

True

In [57]:
(128,60,60,32,3,3),c.kernels.shape

((128, 60, 60, 32, 3, 3), (32, 3, 3, 64))

In [105]:
odip=c.d_inp.forward(outd.get())

In [106]:
cto.shape,odip.shape

((128, 60, 60, 32), (128, 60, 60, 32))

In [107]:
(cto==odip).all()

True

In [109]:
c.backprop(out)

(128, 60, 60, 32)

In [135]:
obb=c.backprop(outd.get())

CPU times: user 3.26 s, sys: 219 ms, total: 3.48 s
Wall time: 1.15 s


In [123]:
np.allclose(odip,obb)

True

In [61]:
c.d_inp.coled.reshape(128,60,60,64,3,3).shape

(128, 60, 60, 64, 3, 3)

In [ ]:
grads=outd

In [166]:
inpd.shape

(128, 32, 60, 60)

In [164]:
kinp=inpd.transpose(1,2,3,0).transpose(0,3,1,2)

In [106]:
grads.shape,inpd.shape,kinp.shape

((128, 60, 60, 64), (128, 32, 60, 60), (32, 128, 60, 60))

In [110]:
ch, kh, kw, nk = grads.shape  # w0d
bt, ch, h, w = kinp.shape    # inpd
sy,sx = (1,1)
ph,pw = (1,1)
dy,dx = (1,1)
out_h,out_w = w0d.shape[1:3]
col = cp.empty((bt, ch, kh, kw, out_h, out_w), dtype=kinp.dtype)

In [111]:
kinp.shape,col.shape,grads.shape

((32, 128, 60, 60), (32, 128, 60, 60, 3, 3), (128, 60, 60, 64))

In [165]:
%%time
kinp=cp.ascontiguousarray(kinp)
cp.cuda.Stream.null.synchronize()

CPU times: user 15 ms, sys: 3.57 ms, total: 18.5 ms
Wall time: 16.8 ms


In [163]:
%%time
col=im2col(kinp.reduced_view(),
              h, w, out_h, out_w, kh, kw, sy, sx, ph, pw, dy, dx, col)
d_ker=cp.tensordot(col, grads, ((1, 2, 3), (0, 1, 2)))
cp.cuda.Stream.null.synchronize()

CPU times: user 452 ms, sys: 2.03 ms, total: 455 ms
Wall time: 453 ms


In [114]:
d_ker.shape

(32, 3, 3, 64)